In [ ]:
# import csv
# import os
# for i in range(12,1200):
#     fname = os.path.join('New_Dataset',str(i)+'.csv')
#     with open(fname,'w',newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(["Image","Annotations"])

In [ ]:
def add_on_csv(idx,Image,GTname):
    for i in idx:
        fname = os.path.join('New_Dataset',str(i)+'.csv')
        with open(fname,'a+',newline='') as file:
            writer = csv.writer(file)
            writer.writerow([Image,GTname])

In [ ]:
from __future__ import print_function, division
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from PIL import Image
import torchvision.transforms.functional as TF

In [ ]:
concepts = ["color","object","part","material","scene","texture"]

class conceptLoader(Dataset):
    """Broaden Dataset Image Loader"""
    
    def __init__(self, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            conceptType (string): type of the concept to load.
        """
        self.c_flag = False  # Concept Flag
        csv_file = os.path.join(root_dir,"index.csv") 
        root_dir = os.path.join(root_dir,"images") 
        self.fileNames = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.fileNames)
    

    def load_concept(self,idx,conceptType):
        c_index = str(self.fileNames.iloc[idx, concepts.index(conceptType)+6])
        image_name = str(self.fileNames.iloc[idx, 0])
        if c_index=='nan':
            self.c_flag = False
            # sample = torch.zeros([1, 113, 113])
            sample =0
        else:
            self.c_flag = True
            if conceptType =="scene" or conceptType =="texture":
                sample  = c_index.split(";") 
            elif conceptType =="part":
                items  = c_index.split(";") 
                sample = []
                c_index=[]
                for i in items:
                    img_name = os.path.join(self.root_dir,i)
                    image = Image.open(img_name)
                    px = np.array(image)
                    item = self.decodeClassMask(px)
                    sample.append(item)
                    c_index.append(i)
                
            else:
                img_name = os.path.join(self.root_dir,c_index)
                image = Image.open(img_name)
                px = np.array(image)
                sample = self.decodeClassMask(px)
                #sample = Image.fromarray(sample)
                #sample = TF.to_tensor(sample)     

        # sample.unsqueeze_(0)
        return self.c_flag, sample,image_name,c_index

    def decodeClassMask(self,im):
        ''' Decodes pixel-level object/part class and instance data from
        the given image, previously encoded into RGB channels.'''
    # Classes are a combination of RG channels
        return (im[:,:,0] + (im[:,:,1])*256)



In [ ]:
    dataset_path='./broden1_227'
    # data_obj = imageLoader(dataset_path)
    data_obj = conceptLoader(dataset_path)
    for j in range(63305):
        flag,aa,ImgName, ConName = data_obj.load_concept(j,"material")
        print(j)
        if flag:
            k=np.unique(aa)
            add_on_csv(k,ImgName,ConName)

In [ ]:
    dataset_path='./broden1_227'
    # data_obj = imageLoader(dataset_path)
    data_obj = conceptLoader(dataset_path)
    for j in range(63305):
        flag,aa,ImgName, ConName = data_obj.load_concept(j,"part")

        if flag:
            for i in range(len(aa)):
                k=np.unique(aa[i])
                add_on_csv(k,ImgName,ConName[i])
        print(j)

In [ ]:
import numpy as np
aa=np.zeros((2,2))
aa[0,0]=1
aa[aa!=1]=0

print(aa)

In [ ]:
import torch 
x=torch.ones([10,3,155,155])

In [ ]:
y=torch.ones([x.shape[0]])

In [ ]:
import os
cwd = os.getcwd()  
dir = os.path.join(cwd,"Tk")
if not os.path.exists(dir):
    os.mkdir(dir)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from skimage.transform import resize
import torchvision.datasets as datasets
from torchvision import models
import matplotlib.pyplot as plt
import time
import numpy as np
import gc

In [ ]:
from torchvision import models
model = models.resnet18(pretrained=True)
name = model.__class__.__name__
print(name)

In [ ]:
os.mkdir('tempDir')

In [ ]:
import numpy as np

In [ ]:
z=np.load('E:\TRDP_II\ICNN\IOU/resnet18/iou_Layer1_conv2d_1.npy')

z.shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot([1, 2, 3, 4])
plt.xlabel('Blah')
plt.ylabel('some numbers')
plt.show()

In [ ]:
plt.plot([1, 2, 3, 4], [1, 4, 9, 16], 'r*')
plt.axis([0, 6, 0, 20])
plt.show()

In [ ]:
import numpy as np

# evenly sampled time at 200ms intervals
t = np.arange(0., 5., 0.2)

# red dashes, blue squares and green triangles
plt.plot(t, t, 'r--', t, t**2, 'bs', t, t**3, 'g^')
plt.show()

In [ ]:
names = ['group_a', 'group_b', 'group_c']
values = [1, 10, 100]

plt.figure(figsize=(9, 3))

plt.subplot(131)
plt.bar(names, values)
plt.subplot(132)
plt.scatter(names, values)
plt.subplot(133)
plt.plot(names, values)
plt.suptitle('Categorical Plotting')
plt.show()

In [ ]:
import numpy as np
from numpy import unravel_index
iou = np.random.randint(10, size=(64,1200,6))
print(iou.shape)

In [ ]:
top = 3
num_concept = 4
# names = ["Tush","Bush","Nush","thush","ghush"]
# values = np.random.randint(10, size=(top,num_concept,5))
names,values = get_concept_summary("resnet18")


for t in range(top):
    plt.figure(figsize=(15, 3))
    plt.suptitle("Top "+str(t+1))

    p1,=plt.plot(names,values[t,0,:],'o--',label="color")
    p2,=plt.plot(names,values[t,1,:],'o--',label="Object")
    p3,=plt.plot(names,values[t,2,:],'o--',label="Part")
    p4,=plt.plot(names,values[t,3,:],'o--',label="Material")

    # Place a legend above this subplot, expanding itself to
    # fully use the given bounding box.
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower right',
           ncol=1, mode="expand", borderaxespad=0.)
    plt.show()
        

In [78]:
def generate_TopThreeIOU(iou,top=3):
    TopThreeIOU = np.zeros([top,iou.shape[0]])
    for u in range(iou.shape[0]):
        U_iou = iou[u,:,:]
        for t in range(top):
            idx =unravel_index(U_iou.argmax(), U_iou.shape)
            TopThreeIOU[t,u]=idx[1]
            U_iou[idx]=0
    return TopThreeIOU

In [79]:
def plot_summary(iou,top=3):

    num_concept = 4 # No. of concepts

    #shape of tp_iou = #top x #No. of units
    tp_iou = generate_TopThreeIOU(iou,top)

    c= np.zeros([top,num_concept])
    for t in range(top):
        temp = np.array(tp_iou[t,:])
        for i in range (num_concept):
            c[t,i] = sum(temp==i)
    return c

In [80]:
import numpy as np
from numpy import unravel_index
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

values = np.zeros((top,num_concept,17))
for layer in range(17):
    iou=np.load('E:\TRDP_II\IOU/resnet18/iou_Layer1_conv2d_1.npy')
    value = plot_summary(iou)
    values[:,:,layer]=value

In [81]:
print(values.shape)

(3, 4, 17)


In [8]:
import numpy as np
r = np.zeros((2,4))
r[1,1] = np.nan
r[np.isnan(r)]=0
print(np.isnan(r))

[[False False False False]
 [False False False False]]


In [14]:
import numpy as np
from numpy import unravel_index
iou = np.random.randint(10, size=(10,3,6))
print(iou)


[[[1 9 2 5 0 7]
  [8 1 9 6 0 9]
  [6 8 0 9 6 0]]

 [[3 7 8 8 2 3]
  [4 0 2 8 2 7]
  [2 1 1 3 1 1]]

 [[9 0 5 4 0 0]
  [7 3 3 3 8 8]
  [6 7 2 5 7 9]]

 [[5 7 8 7 1 9]
  [7 5 9 9 4 6]
  [6 9 7 8 5 0]]

 [[9 7 7 9 4 6]
  [3 4 8 8 5 8]
  [1 0 0 5 7 8]]

 [[5 2 5 2 1 0]
  [6 8 3 3 9 6]
  [0 9 7 1 0 0]]

 [[3 0 2 4 4 4]
  [5 6 9 3 7 3]
  [1 5 2 0 2 2]]

 [[6 2 8 4 5 5]
  [9 5 7 2 2 2]
  [9 8 0 1 5 9]]

 [[0 8 9 0 0 9]
  [5 8 8 9 0 0]
  [0 3 5 8 6 7]]

 [[6 9 5 3 1 3]
  [3 9 7 9 5 2]
  [1 1 7 8 2 8]]]


In [15]:
c_idx = 2
temp = iou[:,c_idx-1,:]

In [18]:
print(temp)
idx =unravel_index(temp.argmax(), temp.shape)
print(idx[0])

[[8 1 9 6 0 9]
 [4 0 2 8 2 7]
 [7 3 3 3 8 8]
 [7 5 9 9 4 6]
 [3 4 8 8 5 8]
 [6 8 3 3 9 6]
 [5 6 9 3 7 3]
 [9 5 7 2 2 2]
 [5 8 8 9 0 0]
 [3 9 7 9 5 2]]
0


In [54]:
def find_top_units(iou,c_idx,top_percentage):

    # Get the number of top units to be computed
    num_top_unit = np.int(np.round(iou.shape[0]*top_percentage/100))
    unit_list = np.zeros((num_top_unit))
    temp = iou[:,c_idx-1,:]
    temp[np.isnan(temp)]=0
    for i in range(num_top_unit):
        idx  = unravel_index(temp.argmax(), temp.shape)
        unit_list[i] = idx[0]
        temp[idx] = 0
    return unit_list

In [55]:
iou=np.load('E:\TRDP_II\IOU/resnet18/iou_Layer1_conv2d_1.npy')
d = find_top_units(iou,121,5)
print(d)

[30. 40. 56.]
